In [49]:
from stock import StockData
from tqdm.notebook import tqdm
import numpy as np
import torch
import pickle as pkl

In [2]:
with open('./stockdata.pkl', 'rb') as f:
    stockdata = pkl.load(f)

In [3]:
stockdata = StockData(reference_data = stockdata.stock_data)

In [6]:
data_frame = stockdata.stock_data

In [59]:
input_code = '155660'
partition = 20

def derive_train_test_tensor(data_frame, partition):
    partition += 1
    
    code_list = set(data_frame['코드'])
        
    for input_code in tqdm(code_list):
        code_data = data_frame[data_frame['코드'] == input_code]

        input_tensor = torch.Tensor()
        output_tensor = torch.Tensor()

        data_length = len(code_data)
        data_access_index = partition

        while data_access_index < data_length:
            # Preprocess
            sample_code = code_data[data_access_index - partition:data_access_index]
            sample_code = np.array(sample_code[['시가', '종가', '저가', '고가', '거래량']]).astype(int)
            input_data = torch.Tensor([sample_code[:partition - 1]])
            output_data = torch.Tensor([sample_code[partition - 1]])

            # Add to whole tensor information
            input_tensor = torch.cat([input_tensor, input_data])
            output_tensor = torch.cat([output_tensor, output_data])

            # Termination
            data_access_index += 1
        
        with open(f'./tensor_training_file/{input_code}.pkl', 'wb') as f:
            pkl.dump({'input': input_tensor, 'output': output_tensor}, f)
            

In [60]:
derive_train_test_tensor(data_frame, 20)